In [ ]:
!pip install \
    tqdm \
    numpy \
    pandas\
    python-dotenv==1.0.0 \
    faiss-cpu \
    unstructured \
    python-magic-bin \
    google-generativeai==0.3.2 \
    langchain \
    langchain-community \
    langchain-core \
    langchain-google-genai==0.0.8 \
    pydantic==1.10.12

In [1]:
import textwrap
import google.generativeai as genai
from IPython.display import display, Markdown
from dotenv import load_dotenv
import os
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredFileLoader
from langchain.vectorstores import FAISS
load_dotenv(".env")
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
from langchain_google_genai import GoogleGenerativeAIEmbeddings

C:\ProgramData\anaconda3\envs\skincare\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
base_path = 'saved_embeddings'
all_dbs = []

gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
for subfolder in sorted(os.listdir(base_path)):
    subfolder_path = os.path.join(base_path, subfolder)
    
    if os.path.isdir(subfolder_path):
        db = FAISS.load_local(folder_path=subfolder_path, embeddings=gemini_embeddings, allow_dangerous_deserialization=True)
        all_dbs.append(db)
        # print(f"Loaded embeddings from {subfolder_path}")

if all_dbs:
    target_db = all_dbs[0]

    # Merge all other databases into the target database
    for db in all_dbs[1:]:
        FAISS.merge_from(target_db, db)
    
    retriever = target_db.as_retriever(search_kwargs={'k': 3})
else:
    print("No databases loaded.")

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0.3, top_p=0.85)

from langchain.chains import RetrievalQAWithSourcesChain
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=retriever)

from langchain import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain.schema.runnable import RunnablePassthrough



llm_prompt_template = """You are an assistant for generating Ayurvedic skincare formulas based on the provided context.
Use the following context to create a personalized skincare formula.
If you don't know the answer, try to think of it without context.\n
Question: {question} \nContext: {context} \nAnswer:"""

llm_prompt = PromptTemplate.from_template(llm_prompt_template)

print(llm_prompt)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | llm_prompt
    | llm
    | StrOutputParser()
)

# Invoke the chain with example questions
responses = [
    rag_chain.invoke("Create a skincare formula for dry skin"),
    rag_chain.invoke("Suggest an Ayurvedic remedy for acne-prone skin"),
    rag_chain.invoke("What ingredients should I use for glowing skin?"),
    rag_chain.invoke("How can I reduce dark spots using Ayurvedic methods?"),
    rag_chain.invoke("Recommend a skincare routine for oily skin"),
    rag_chain.invoke("What Ayurvedic herbs are best for anti-aging?"),
    rag_chain.invoke("I have sensitive skin. What skincare formula should I use?")
]

# Print the responses
for response in responses:
    print(response)

input_variables=['context', 'question'] template="You are an assistant for generating Ayurvedic skincare formulas based on the provided context.\nUse the following context to create a personalized skincare formula.\nIf you don't know the answer, try to think of it without context.\n\nQuestion: {question} \nContext: {context} \nAnswer:"
Based on the provided text, here's an Ayurvedic-inspired skincare formula for dry skin, focusing on readily available and traditionally used ingredients:

**Name:**  Nourishing Ayurvedic Dry Skin Cream

**Ingredients:**

* **Shea Butter (20%):**  A rich emollient, deeply moisturizing and protective.  Acts as a base.
* **Almond Oil (15%):** Excellent emollient, softens and nourishes dry skin.
* **Coconut Oil (10%):**  Adds moisture and creates a smooth texture.  (Use fractionated coconut oil for a lighter feel if desired).
* **Aloe Vera Gel (10%):** Soothes and hydrates, promotes skin healing.
* **Glycerin (5%):** Humectant, draws moisture from the air to

In [ ]:
while True:
    question=input("Ask a question:")
    if "stop" in question:
        break
    print(question+"\n\n")
    response=rag_chain.invoke(question)
    print(response+"\n\n")